DATARESEARCH2: EN este documento nos centraremos en obtener tablas importantes para nuestra seleccion de paises, la inflacion, o deflactor del PIB, es un factor precursor de politicas negativas para el bienestar del individuo,
tambien se puede relacionar con el PIB PER CAPITA, el SMI y el Sueldo bruto Anual Medio.
Vamos a relacionar datos economicos con otros factores importantes a la hora de agrupar paises como la zona geografica y similitudes culturales.
Voy a utilizar Selenium com ChromeDriver instalado en mi ordenador de sobremesa, pero ahora mismo hay una extension de chrome , en la web de chrome de extensiones llamada "Instant Data Scrapper" que ofrece la posibilidad de descargar las tablas en formato
Excel y CSV, a tiempo real, sin ningun tipo de explorador configurado para tareas automatizadas.

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.service import Service 
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from bs4 import BeautifulSoup as bs
import requests
import random 
import pandas as pd
import numpy as np
import sqlite3
import time
import os

ModuleNotFoundError: No module named 'webdriver_manager'

In [ ]:
#Procedemos a descargar la tabla de Inflacion en la zona euro del 2023, dado que la del año 2024 aun no esta disponible.
driver = webdriver.Chrome()
url = "https://datos.bancomundial.org/indicador/NY.GDP.DEFL.KD.ZG?contextual=region&end=2023&locations=EU-DK&most_recent_value_desc=false&start=1961&view=chart"
#Nuestra tabla esta anidada en un div de esta forma:
#<div><div class="item" style="min-height: 58px;"><div data-customlink="nl:body content" data-text="Dinamarca">ent_value<a class="country-name" 
#href="/indicador/NY.GDP.DEFL.KD.ZG?contextual=region&amp;end=2023&amp;locations=EU-DK&amp;most_rec_desc=false&amp;start=1961&amp;view=chart">Dinamarca</a></div><div>2023</div><div>-3,8</div><div>
'''Como vemos, encontramos los paises dentro de un div por el class name de country-name y los datos de año e inflacion en dos items div posteriormente'''
driver.get(url)
# XPath para encontrar todos los elementos con la clase 'item'
xpath_items = "//div[@class='item']"
items = driver.find_elements(By.XPATH, xpath_items)
data = []
for item in items:
    # Extraer el nombre del país
    pais = item.find_element(By.XPATH, ".//a[@class='country-name']").text.strip()

    # Extraer el año
    ano = item.find_elements(By.TAG_NAME, "div")[1].text.strip() 

    # Extraer el valor de la inflación
    inflacion = item.find_elements(By.TAG_NAME, "div")[2].text.strip() 

    data.append({"País": pais, "Año": ano, "Inflación": inflacion})

# Crear un DataFrame de pandas
import pandas as pd
df_inflacion = pd.DataFrame(data)
print(df_inflacion)

In [8]:
# Crear la tabla si no existe
conn = sqlite3.connect('C:/GitHubRepos/ProyectoFinal/data/processed/Dataset9k.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS EuroInfl (
        Pais TEXT,
        Año INTEGER,
        Inflacion REAL
    )
''')

# Insertar el DataFrame en la tabla sin el índice
df_inflacion.to_sql('EuroInfl', conn, if_exists='replace', index=False)

# Cerrar la conexión a la base de datos
conn.close()